In [24]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of models.flow_module failed: Traceback (most recent call last):
  File "/data/rsg/chemistry/jyim/mambaforge/envs/fm/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/data/rsg/chemistry/jyim/mambaforge/envs/fm/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/data/rsg/chemistry/jyim/mambaforge/envs/fm/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/data/rsg/chemistry/jyim/mambaforge/envs/fm/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/data/rsg/chemistry/jyim/mambaforge/envs/fm/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/data/rsg/chemistry/jyim/mambaforge/envs/fm/lib/pyth

In [25]:
import os
from omegaconf import OmegaConf
from models.flow_module import FlowModule
import torch
from data.pdb_dataloader import PdbDataModule
import GPUtil
from data import utils as du
import numpy as np
import tree
from data import so3_utils
from data import all_atom
from analysis import utils as au
from openfold.utils.superimposition import superimpose
import matplotlib.pyplot as plt
import copy


In [26]:
# Setup lightning module

ckpt_dir = '../ckpt/se3-fm/refactor_with_align/2023-10-15_18-12-18/'
ckpt_path = os.path.join(ckpt_dir, 'last.ckpt')

print(ckpt_path)

base_path = '../configs/base.yaml'
base_cfg = OmegaConf.load(base_path)

cfg_path = os.path.join(ckpt_dir, 'config.yaml')
ckpt_cfg = OmegaConf.load(cfg_path)

OmegaConf.set_struct(base_cfg, False)
OmegaConf.set_struct(ckpt_cfg, False)
cfg = OmegaConf.merge(base_cfg, ckpt_cfg)
cfg.experiment.checkpointer.dirpath = './'
cfg.experiment.rescale_time = False
cfg.data.dataset.csv_path = '../preprocessed/metadata.csv'
cfg.data.loader.num_workers = 0
device = f'cpu'

../ckpt/se3-fm/refactor_with_align/2023-10-15_18-12-18/last.ckpt


In [27]:
# Set up data module
data_module = PdbDataModule(cfg.data)
data_module.setup('fit')

# Test that each rank gets own set of indices

In [24]:
def indices(rank):
    train_dataloader = data_module.train_dataloader(
        num_replicas=2,
        rank=rank
    )
    data_iter = iter(train_dataloader)
    all_csv_idx = []
    for batch in data_iter:
        all_csv_idx.append(batch['csv_idx'])
    all_csv_idx = torch.concatenate(all_csv_idx).squeeze()
    return sorted(all_csv_idx.unique().tolist())

In [25]:
rank_0_indices = indices(0)
rank_1_indices = indices(1)

In [28]:
intersection_indices = set(rank_1_indices) & set(rank_0_indices)

In [34]:
train_dataloader = data_module.train_dataloader(
    num_replicas=2,
    rank=0
)
raw_csv = train_dataloader.dataset.csv

In [38]:
all_indices = raw_csv.index.tolist()

In [39]:
len(all_indices)

3938

In [40]:
len(set(rank_0_indices) | set(rank_1_indices))

3938

# Test that each epoch changes the batch order

In [5]:
train_dataloader = data_module.train_dataloader(
    num_replicas=2,
    rank=0,
)
data_iter_1 = iter(train_dataloader)
# print(train_dataloader.batch_sampler.epoch)
data_iter_2 = iter(train_dataloader)
# print(train_dataloader.batch_sampler.epoch)

In [9]:
batch_1 = next(data_iter_1)
idx_1 = batch_1['csv_idx']
res_1 = batch_1['res_mask'].shape[1]

batch_2 = next(data_iter_2)
idx_2 = batch_2['csv_idx']
res_2 = batch_2['res_mask'].shape[1]
while res_2 != res_1:
    batch_2 = next(data_iter_2)
    idx_2 = batch_2['csv_idx']
    res_2 = batch_2['res_mask'].shape[1]    

In [18]:
len(set(idx_1.squeeze().numpy()) & set(idx_2.squeeze().numpy()))

17

In [17]:
len(idx_1)

27

# Test BatchOT

In [28]:
train_dataloader = data_module.train_dataloader(
    num_replicas=2,
    rank=0
)
data_iter = iter(train_dataloader)

In [29]:
batch = next(data_iter)

In [38]:
from scipy.optimize import linear_sum_assignment

In [40]:
trans_1 = batch['trans_1']
res_mask = batch['res_mask']
trans_0 = torch.randn(trans_1.shape) * 10.0
num_batch, num_res = res_mask.shape

noise_idx, gt_idx = torch.where(
    torch.ones(num_batch, num_batch))
batch_nm_0 = trans_0[noise_idx]
batch_nm_1 = trans_1[gt_idx]
batch_mask = res_mask[gt_idx]
aligned_nm_0, aligned_nm_1, _ = du.batch_align_structures(
    batch_nm_0, batch_nm_1, mask=batch_mask
) 

aligned_nm_0 = aligned_nm_0.reshape(num_batch, num_batch, num_res, 3)
aligned_nm_1 = aligned_nm_1.reshape(num_batch, num_batch, num_res, 3)

# Compute cost matrix of aligned noise to ground truth
batch_mask = batch_mask.reshape(num_batch, num_batch, num_res)
cost_matrix = torch.sum(
    torch.linalg.norm(aligned_nm_0 - aligned_nm_1, dim=-1), dim=-1
) / torch.sum(batch_mask, dim=-1)
noise_perm, gt_perm = linear_sum_assignment(du.to_numpy(cost_matrix))
aligned_trans_0 = aligned_nm_0[(tuple(gt_perm), tuple(noise_perm))]

In [42]:
trans_dist = torch.mean(torch.linalg.norm(trans_0 - trans_1, dim=-1), -1)

In [43]:
aligned_dist = torch.mean(torch.linalg.norm(aligned_trans_0 - trans_1, dim=-1), -1)

In [46]:
torch.mean(trans_dist)

tensor(19.7388)

In [47]:
torch.mean(aligned_dist)

tensor(18.3862)

In [ ]:
num_batch, num_res = trans_0.shape[:2]
noise_idx, gt_idx = torch.where(
    torch.ones(num_batch, num_batch))
batch_nm_0 = trans_0[noise_idx]
batch_nm_1 = trans_1[gt_idx]
batch_mask = res_mask[gt_idx]
aligned_nm_0, aligned_nm_1, _ = du.batch_align_structures(
    batch_nm_0, batch_nm_1, mask=batch_mask
) 
aligned_nm_0 = aligned_nm_0.reshape(num_batch, num_batch, num_res, 3)
aligned_nm_1 = aligned_nm_1.reshape(num_batch, num_batch, num_res, 3)

# Compute cost matrix of aligned noise to ground truth
batch_mask = batch_mask.reshape(num_batch, num_batch, num_res)
cost_matrix = torch.sum(
    torch.linalg.norm(aligned_nm_0 - aligned_nm_1, dim=-1), dim=-1
) / torch.sum(batch_mask, dim=-1)
noise_perm, gt_perm = linear_sum_assignment(du.to_numpy(cost_matrix))
return aligned_nm_0[(tuple(gt_perm), tuple(noise_perm))]

In [ ]:
    def _batch_ot(self, trans_0, trans_1, res_mask):
        num_batch, num_res = trans_0.shape[:2]
        noise_idx, gt_idx = torch.where(
            torch.ones(num_batch, num_batch))
        batch_nm_0 = trans_0[noise_idx]
        batch_nm_1 = trans_1[gt_idx]
        batch_mask = res_mask[gt_idx]
        aligned_nm_0, aligned_nm_1, _ = du.batch_align_structures(
            batch_nm_0, batch_nm_1, mask=batch_mask
        ) 
        aligned_nm_0 = aligned_nm_0.reshape(num_batch, num_batch, num_res, 3)
        aligned_nm_1 = aligned_nm_1.reshape(num_batch, num_batch, num_res, 3)
        
        # Compute cost matrix of aligned noise to ground truth
        batch_mask = batch_mask.reshape(num_batch, num_batch, num_res)
        cost_matrix = torch.sum(
            torch.linalg.norm(aligned_nm_0 - aligned_nm_1, dim=-1), dim=-1
        ) / torch.sum(batch_mask, dim=-1)
        noise_perm, gt_perm = linear_sum_assignment(du.to_numpy(cost_matrix))
        return aligned_nm_0[(tuple(gt_perm), tuple(noise_perm))]